In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm


In [ ]:
df = pd.read_excel("/content/Vueling.xlsx", sheet_name="Unconstrained Demand")

In [ ]:
df

,Date (MM/DD/YYYY),Classic demand after holiday,Eco Demand
0,2020-12-31,130,273
1,2021-01-01,147,221
2,2021-01-02,145,218
3,2021-01-03,151,227
4,2021-01-04,118,224
...,...,...,...
1567,2025-04-16,96,154
1568,2025-04-17,103,175
1569,2025-04-18,95,143
1570,2025-04-19,115,253


In [ ]:
df.columns.values[0] = 'date'
df = df.rename(columns={df.columns[0]: 'date', df.columns[1]: "Classic", df.columns[2]: "Eco"})

df['date'] = pd.to_datetime(df['date'])

# Extract day of week, month (as number or name), and year
df['day'] = df['date'].dt.day_name()  # Monday, Tuesday, etc.
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year




In [ ]:
df

,date,Classic,Eco,day,month,year
0,2020-12-31,130,273,Thursday,12,2020
1,2021-01-01,147,221,Friday,1,2021
2,2021-01-02,145,218,Saturday,1,2021
3,2021-01-03,151,227,Sunday,1,2021
4,2021-01-04,118,224,Monday,1,2021
...,...,...,...,...,...,...
1567,2025-04-16,96,154,Wednesday,4,2025
1568,2025-04-17,103,175,Thursday,4,2025
1569,2025-04-18,95,143,Friday,4,2025
1570,2025-04-19,115,253,Saturday,4,2025


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1572 entries, 0 to 1571
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   date     1572 non-null   datetime64[ns]
 1   Classic  1572 non-null   int64         
 2   Eco      1572 non-null   int64         
 3   day      1572 non-null   object        
 4   month    1572 non-null   int32         
 5   year     1572 non-null   int32         
dtypes: datetime64[ns](1), int32(2), int64(2), object(1)
memory usage: 61.5+ KB


In [ ]:
day_dummy = pd.get_dummies(df['day'], prefix='day').astype(int)
month_dummy = pd.get_dummies(df['month'], prefix='month').astype(int)
year_dummy = pd.get_dummies(df['year'], prefix='year').astype(int)

df= df.drop('date', axis = 1)
revised_df = pd.concat([df, day_dummy, month_dummy, year_dummy], axis=1)
revised_df.head()


,Classic,Eco,day,month,year,day_Friday,day_Monday,day_Saturday,day_Sunday,day_Thursday,...,month_9,month_10,month_11,month_12,year_2020,year_2021,year_2022,year_2023,year_2024,year_2025
0,130,273,Thursday,12,2020,0,0,0,0,1,...,0,0,0,1,1,0,0,0,0,0
1,147,221,Friday,1,2021,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,145,218,Saturday,1,2021,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,151,227,Sunday,1,2021,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
4,118,224,Monday,1,2021,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [ ]:
Y_classic = revised_df['Classic']
Y_eco= revised_df['Eco']
X = revised_df.drop(['Classic','Eco','day','month','year'], axis=1)
X= sm.add_constant(X)


In [ ]:
X

,const,day_Friday,day_Monday,day_Saturday,day_Sunday,day_Thursday,day_Tuesday,day_Wednesday,month_1,month_2,...,month_9,month_10,month_11,month_12,year_2020,year_2021,year_2022,year_2023,year_2024,year_2025
0,1.0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0
1,1.0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
2,1.0,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3,1.0,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
4,1.0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1567,1.0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
1568,1.0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1569,1.0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1570,1.0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
model_classic = sm.OLS(Y_classic, X).fit()
model_classic.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                Classic   R-squared:                       0.853
Model:                            OLS   Adj. R-squared:                  0.851
Method:                 Least Squares   F-statistic:                     409.6
Date:                Sun, 27 Apr 2025   Prob (F-statistic):               0.00
Time:                        13:57:11   Log-Likelihood:                -5845.1
No. Observations:                1572   AIC:                         1.174e+04
Df Residuals:                    1549   BIC:                         1.186e+04
Df Model:                          22                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const            75.8580      1.220     62.170      0.000      73.465      78.251
day_Friday       15.6227      0.646     24.193      0.000      14.356      16.889
day_Monday        7.1243      0.647     11.014      0.000       5.856       8.393
day_Saturday     15.6235      0.646     24.198      0.000      14.357      16.890
day_Sunday        7.3626      0.646     11.403      0.000       6.096       8.629
day_Thursday     14.2665      0.635     22.475      0.000      13.021      15.512
day_Tuesday       8.0954      0.647     12.516      0.000       6.827       9.364
day_Wednesday     7.7628      0.647     12.003      0.000       6.494       9.031
month_1          34.8681      0.798     43.693      0.000      33.303      36.433
month_2          32.9445      0.831     39.633      0.000      31.314      34.575
month_3          -5.3676      0.798     -6.726      0.000      -6.933      -3.802
month_4          -6.4954      0.826     -7.863      0.000      -8.116      -4.875
month_5          -6.7045      0.872     -7.693      0.000      -8.414      -4.995
month_6         -26.8452      0.885    -30.347      0.000     -28.580     -25.110
month_7         -25.9536      0.872    -29.779      0.000     -27.663     -24.244
month_8         -25.7761      0.872    -29.574      0.000     -27.486     -24.066
month_9          22.0390      0.885     24.914      0.000      20.304      23.774
month_10         25.0366      0.872     28.727      0.000      23.327      26.746
month_11         23.8248      0.885     26.934      0.000      22.090      25.560
month_12         34.2872      0.862     39.766      0.000      32.596      35.978
year_2020         5.5883      8.619      0.648      0.517     -11.318      22.495
year_2021        14.3490      1.560      9.198      0.000      11.289      17.409
year_2022        14.4120      1.560      9.238      0.000      11.352      17.472
year_2023        14.2894      1.560      9.160      0.000      11.229      17.349
year_2024        13.8824      1.560      8.900      0.000      10.823      16.942
year_2025        13.3369      1.734      7.690      0.000       9.935      16.739
==============================================================================
Omnibus:                        3.963   Durbin-Watson:                   2.042
Prob(Omnibus):                  0.138   Jarque-Bera (JB):                3.913
Skew:                          -0.096   Prob(JB):                        0.141
Kurtosis:                       2.850   Cond. No.                     6.18e+15
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 5.96e-29. This might indicate that there are
strong multicollinearity problems or that the design ma

In [ ]:
def backward_elimination(X, y, significance_level=0.05):
    X = sm.add_constant(X)
    variables = list(X.columns)

    while True:
        model = sm.OLS(y, X[variables]).fit()
        p_values = model.pvalues.iloc[1:]  # Skip intercept

        max_pval = p_values.max()
        if max_pval > significance_level:
            excluded_var = p_values.idxmax()
            print(f"Dropping '{excluded_var}' with p-value {max_pval:.4f}")
            variables.remove(excluded_var)
        else:
            break

    final_model = sm.OLS(y, X[variables]).fit()
    return final_model



In [ ]:
final_model_classic = backward_elimination(X, Y_classic)
print(final_model_classic.summary())

Dropping 'year_2020' with p-value 0.5169
Dropping 'year_2025' with p-value 0.4456
Dropping 'year_2024' with p-value 0.5839
Dropping 'year_2023' with p-value 0.4442
Dropping 'year_2021' with p-value 0.5682
Dropping 'year_2022' with p-value 0.6024
                            OLS Regression Results                            
Dep. Variable:                Classic   R-squared:                       0.853
Model:                            OLS   Adj. R-squared:                  0.852
Method:                 Least Squares   F-statistic:                     531.0
Date:                Sun, 27 Apr 2025   Prob (F-statistic):               0.00
Time:                        13:57:11   Log-Likelihood:                -5846.1
No. Observations:                1572   AIC:                         1.173e+04
Df Residuals:                    1554   BIC:                         1.182e+04
Df Model:                          17                                         
Covariance Type:            nonrobust      

In [ ]:
dates_to_predict = pd.date_range("2025-06-17", "2025-06-23")

predict_df = pd.DataFrame({'date': dates_to_predict})
predict_df['day'] = predict_df['date'].dt.day_name()
predict_df['month'] = predict_df['date'].dt.month
predict_df['year'] = predict_df['date'].dt.year

print(predict_df)

        date        day  month  year
0 2025-06-17    Tuesday      6  2025
1 2025-06-18  Wednesday      6  2025
2 2025-06-19   Thursday      6  2025
3 2025-06-20     Friday      6  2025
4 2025-06-21   Saturday      6  2025
5 2025-06-22     Sunday      6  2025
6 2025-06-23     Monday      6  2025


In [ ]:
coefficients = final_model_classic.params
std_errors = final_model_classic.bse
print(std_errors)

const            0.207229
day_Friday       0.619996
day_Monday       0.621214
day_Saturday     0.619986
day_Sunday       0.620000
day_Thursday     0.620002
day_Tuesday      0.621203
day_Wednesday    0.621201
month_1          0.777507
month_2          0.811743
month_3          0.777482
month_4          0.814432
month_5          0.861199
month_6          0.874350
month_7          0.861191
month_8          0.861197
month_9          0.874348
month_10         0.861194
month_11         0.874355
month_12         0.858019
dtype: float64


In [ ]:
mean=[]
std = []

for index, row in predict_df.iterrows():
    mean_value = coefficients[f"day_{row['day']}"] + coefficients[f"month_{row['month']}"] + coefficients["const"]
    mean.append(mean_value)

for index, row in predict_df.iterrows():
    std_value = (std_errors[f"day_{row['day']}"]**2 + std_errors[f"month_{row['month']}"]**2 + std_errors["const"]**2)**(1/2)
    std.append(std_value)

In [ ]:
mean

[np.float64(71.34806193925532),
 np.float64(71.01319372235534),
 np.float64(77.47891230893023),
 np.float64(78.87460995365143),
 np.float64(78.87538834763201),
 np.float64(70.61417631848599),
 np.float64(70.37712937084063)]

In [ ]:
std

[np.float64(1.0923945857505635),
 np.float64(1.0923931637383864),
 np.float64(1.0917119138724634),
 np.float64(1.0917081449258619),
 np.float64(1.0917025370429925),
 np.float64(1.0917106289306482),
 np.float64(1.0924005861838124)]

In [ ]:

results_df = pd.DataFrame({
    'Date': predict_df['date'],
    'Predicted_Mean': mean,
    'Standard_Deviation': std
})

results_df.to_excel('prediction_results.xlsx', index=False)


In [ ]:
Y_pred = final_model_classic.fittedvalues

# Residuals (difference between actual and predicted values)
residuals = Y_classic - Y_pred

# Standard error of the regression (residual standard error)
n = len(Y_classic)  # Number of observations
k = len(X.columns)  # Number of parameters (including the intercept)

# Calculate the residual sum of squares
RSS = (residuals ** 2).sum()

# Calculate the standard error of the regression
standard_error_of_regression = (RSS / (n - k)) ** 0.5
print(f"Standard Error of the regression: {standard_error_of_regression}")

Standard Error of the regression: 10.057332015493895
